# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [9]:
# import libraries
import numpy as np
import pandas as pd
from sqlalchemy import create_engine
import re

import nltk
nltk.download(['punkt','wordnet', 'averaged_perceptron_tagger'])
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer

from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, precision_score, \
recall_score, f1_score
from sklearn.base import BaseEstimator, TransformerMixin

import pickle

[nltk_data] <urlopen error [Errno 61] Connection refused>


In [10]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql_table('DisasterResponse', engine)
X = df.message.values
y = df.iloc[:, 4:].values

OperationalError: (sqlite3.OperationalError) no such table: InsertDatabaseName
[SQL: SELECT * FROM InsertDatabaseName]
(Background on this error at: http://sqlalche.me/e/e3q8)

### 2. Write a tokenization function to process your text data

In [3]:
url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
def tokenize(text):
    # get list of all urls using regex
    detected_urls = re.findall(url_regex, text)
    
    # replace each url in text string with placeholder
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")
    
    # tokenize text
    tokens = word_tokenize(text)
    
    # initiate lemmatizer
    lemmatizer = WordNetLemmatizer()
    
    # iterate through each token
    clean_tokens = []
    for tok in tokens:
        
        # lemmatize, normalize case, and remove leading/trailing white space
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)
    
    return clean_tokens
        

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

# train classifier
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [6]:
# predict on test data
y_pred = pipeline.predict(X_test)


In [7]:
def display_results(y_test, y_pred):
    categories = df.iloc[:, 4:].columns.values
    for i in range(0, len(categories)):
        print('--------------------\n')
        print('Category: {}\n'.format(categories[i]))
        print(classification_report(y_test[:,i],y_pred[:,i]))


In [8]:
display_results(y_test, y_pred)

--------------------

Category: related

             precision    recall  f1-score   support

          0       0.63      0.37      0.46      1502
          1       0.82      0.93      0.87      4994
          2       0.41      0.16      0.23        58

avg / total       0.77      0.80      0.77      6554

--------------------

Category: request

             precision    recall  f1-score   support

          0       0.88      0.99      0.93      5408
          1       0.85      0.38      0.53      1146

avg / total       0.88      0.88      0.86      6554

--------------------

Category: offer

             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6523
          1       0.00      0.00      0.00        31

avg / total       0.99      1.00      0.99      6554

--------------------

Category: aid_related

             precision    recall  f1-score   support

          0       0.70      0.88      0.78      3698
          1       0.77      0

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
# specify parameters for grid search
parameters = {
#         'vect__ngram_range': ((1, 1), (1, 2)),
#         'vect__max_df': (0.75, 1.0),
#         'vect__max_features': (None, 5000),
#         'tfidf__use_idf': (True, False),
        'clf__estimator__n_estimators': [50, 100],
#         'clf__estimator__min_samples_split': [2, 3]
}

# create grid search object
cv =  GridSearchCV(pipeline, param_grid=parameters, verbose = 2)

# train classifier
cv.fit(X_train, y_train)

Fitting 3 folds for each of 4 candidates, totalling 12 fits
[CV] clf__estimator__n_estimators=50, vect__max_df=0.75 ..............
[CV]  clf__estimator__n_estimators=50, vect__max_df=0.75, total= 3.1min
[CV] clf__estimator__n_estimators=50, vect__max_df=0.75 ..............


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  3.5min remaining:    0.0s


[CV]  clf__estimator__n_estimators=50, vect__max_df=0.75, total= 3.1min
[CV] clf__estimator__n_estimators=50, vect__max_df=0.75 ..............
[CV]  clf__estimator__n_estimators=50, vect__max_df=0.75, total= 3.1min
[CV] clf__estimator__n_estimators=50, vect__max_df=1.0 ...............
[CV]  clf__estimator__n_estimators=50, vect__max_df=1.0, total= 3.1min
[CV] clf__estimator__n_estimators=50, vect__max_df=1.0 ...............
[CV]  clf__estimator__n_estimators=50, vect__max_df=1.0, total= 3.1min
[CV] clf__estimator__n_estimators=50, vect__max_df=1.0 ...............
[CV]  clf__estimator__n_estimators=50, vect__max_df=1.0, total= 3.1min
[CV] clf__estimator__n_estimators=100, vect__max_df=0.75 .............


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
# predict on test data
y_pred = pipeline.predict(X_test)

In [ ]:
display_results(y_test, y_pred)

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [4]:
# create a custom transformer
class StartingVerbExtractor(BaseEstimator, TransformerMixin):

    def starting_verb(self, text):
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            first_word, first_tag = pos_tags[0]
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True
        return False

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)

In [7]:
pipeline = Pipeline([
        ('features', FeatureUnion([

            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ])),

            ('starting_verb', StartingVerbExtractor())
        ])),

        ('clf', MultiOutputClassifier(AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=1))))    
])

parameters = {
#         'features__text_pipeline__vect__ngram_range': ((1, 1), (1, 2)),
        'features__text_pipeline__vect__max_df': (0.75, 1.0),
#         'features__text_pipeline__vect__max_features': (None, 5000),
#         'features__text_pipeline__tfidf__use_idf': (True, False),
#         'clf__estimator__n_estimators': [50, 80],
        'clf__estimator__learning_rate': [0.1, 0.3]
#         'features__transformer_weights': (
#             {'text_pipeline': 1, 'starting_verb': 0.5},
#             {'text_pipeline': 0.5, 'starting_verb': 1},
#             {'text_pipeline': 0.8, 'starting_verb': 1}    
#         )
}

# create grid search object
cv_improved = GridSearchCV(pipeline, param_grid = parameters, verbose=2)

# train classifier
cv_improved.fit(X_train, y_train)

NameError: name 'tokenize' is not defined

In [ ]:
# predict on test data
y_pred = pipeline.predict(X_test)

In [ ]:
display_results(y_test, y_pred)

### 9. Export your model as a pickle file

In [ ]:
with open('response_cv.pkl', 'wb') as file:
    pickle.dump(cv, file)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.